## 1. *Band-pass Filter Parameters*
- `BAND_LO` and `BAND_HI`: Adjust frequency range (currently 12-18 kHz)
- Target different frequency bands for different battery states

    - This is essentially how we filter only the frequencies within our range of interest (the ultrasonic waves)

## 2. *Spectral Subtraction Parameters*
- `QUIET_PCT`: Percentage of quietest frames for noise estimation (currently 20%)
- `OVERSUB`: Over-subtraction factor (currently 1.2)
    - used to remove noise from our recordings

## 3. **STFT Parameters**
- `NFFT`: FFT size affects frequency resolution (currently 2048)
- `HOP`: Hop length affects time resolution (currently 512)

    - an STFT is basically just a time frequency domain representation of the audio waveform, instead of showing up in a magnitude format. 

## 4. **Mel-spectrogram Parameters**
- `N_MELS`: Number of mel bins (currently 64)
- `n_fft`: FFT size for mel transform (currently 1024)

    - Mel-spectrograms are a way to represent audio signals in a way that aligns with how we perceive sound, specifically focusing on freqs that we can hear more distinctly.

In [ ]:
import os, glob, subprocess, math
import numpy as np
from PIL import Image

import torch, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models

import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as AF

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from collections import Counter

# SETTINGS
RAW_VIDEOS   = "raw_videos"     # input videos
AUDIO_DIR    = "audio_segments"   # output 2s wavs
IMAGE_DIR    = "recorded_images"  # output frame jpgs

SAMPLE_RATE  = 48000       
CHUNK_SECONDS= 2
N_MELS       = 64
BATCH_SIZE   = 20
EPOCHS       = 15
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

label_map = {"full": 0, "half": 1, "empty": 2}
LABELS = {v: k.capitalize() for k, v in label_map.items()}

os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

In [ ]:
# EXTRACT 2s AUDIO + FRAMES FROM VIDEO
def extract_audio(video_path, wav_out, sample_rate=SAMPLE_RATE):
    cmd = [
        "ffmpeg", "-y", "-i", video_path,
        "-ar", str(sample_rate), "-ac", "1", wav_out
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

def extract_frame(video_path, image_out, timestamp):
    cmd = ["ffmpeg", "-y", "-i", video_path, "-ss", f"{timestamp:.3f}", "-vframes", "1", image_out]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

def split_audio_and_frames(video_path, audio_dir=AUDIO_DIR, image_dir=IMAGE_DIR,
                           chunk_sec=CHUNK_SECONDS, sample_rate=SAMPLE_RATE):
    base = os.path.splitext(os.path.basename(video_path))[0]   # e.g., full_8
    # 1) extract full audio
    full_audio = os.path.join(audio_dir, base + ".wav")
    extract_audio(video_path, full_audio, sample_rate=sample_rate)

    # 2) load and split into 2s chunks
    waveform, sr = torchaudio.load(full_audio)   # [1,T]
    if sr != sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, sample_rate)
        sr = sample_rate

    total_samples = waveform.shape[-1]
    chunk_samples = int(chunk_sec * sr)
    n_chunks = total_samples // chunk_samples

    for i in range(n_chunks):
        start = i * chunk_samples
        end   = start + chunk_samples
        seg   = waveform[:, start:end]                  
        seg_name = f"{base}_seg{i}.wav"
        seg_path = os.path.join(audio_dir, seg_name)
        torchaudio.save(seg_path, seg, sr)

        # frame at middle of window (i*2s + 1.0s)
        ts = (i * chunk_sec) + (chunk_sec / 2.0)
        frame_name = f"{base}_frame{i}.jpg"
        frame_path = os.path.join(image_dir, frame_name)
        extract_frame(video_path, frame_path, ts)

    print(f"[INFO] {base}: wrote {n_chunks} segments")

def prepare_dataset_from_videos(raw_videos=RAW_VIDEOS):
    video_files = []
    video_files.extend(glob.glob(os.path.join(raw_videos, "*.MOV")))
    video_files.extend(glob.glob(os.path.join(raw_videos, "*.mov")))
    video_files.extend(glob.glob(os.path.join(raw_videos, "*.mp4")))
    print(video_files)
    if not video_files:
        print(f"[WARN] No videos in {raw_videos}")
    for vf in video_files:
        split_audio_and_frames(vf)

In [ ]:
# IMAGE TRANSFORM
gray = transforms.Grayscale(num_output_channels=3)
transform_image = transforms.Compose([
    transforms.Resize((224,224)),
    gray,
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# Audio cleaning for a 2s chunk (band-pass + quiet-frame spectral subtraction)
SR        = SAMPLE_RATE
BAND_LO   = 12000
BAND_HI   = 18000
NFFT      = 2048
HOP       = 512
OVERSUB   = 1.2
QUIET_PCT = 0.20

def _stft(x: torch.Tensor) -> torch.Tensor:
    if x.dim() == 2: x = x.squeeze(0)
    return torch.stft(x, n_fft=NFFT, hop_length=HOP, win_length=NFFT,
                      window=torch.hann_window(NFFT), return_complex=True, center=True)

def _istft(S: torch.Tensor, length: int) -> torch.Tensor:
    """
    Inverse STFT that accepts complex tensors when supported,
    and falls back to view_as_real format otherwise.
    Always uses a real-valued Hann window (float32).
    """
    win = torch.hann_window(NFFT, device=S.device, dtype=torch.float32)
    try:
        y = torch.istft(
            S, n_fft=NFFT, hop_length=HOP, win_length=NFFT,
            window=win, length=length, center=True
        )
    except (TypeError, RuntimeError):
        y = torch.istft(
            torch.view_as_real(S), n_fft=NFFT, hop_length=HOP, win_length=NFFT,
            window=win, length=length, center=True
        )
    return y

def bandpass_chunk(waveform: torch.Tensor, sr: int) -> torch.Tensor:
    """
    Band-pass in STFT domain by zeroing bins outside [BAND_LO, BAND_HI].
    Version-agnostic; avoids torchaudio biquad/lfilter kernels.
    Input:  waveform [1, T] (float32)
    Output: [1, T]
    """
    if waveform.dim() == 1:
        waveform = waveform.unsqueeze(0)
    x = waveform.squeeze(0).to(torch.float32)          # [T]

    # STFT
    S = torch.stft(x, n_fft=NFFT, hop_length=HOP, win_length=NFFT,
                   window=torch.hann_window(NFFT, dtype=torch.float32),
                   return_complex=True, center=True)   # [F, T], complex

    # Frequency mask
    freqs = np.fft.rfftfreq(NFFT, d=1.0/sr)
    lo = int(np.searchsorted(freqs, BAND_LO))
    hi = int(np.searchsorted(freqs, BAND_HI))
    lo = max(lo, 0); hi = min(hi, S.shape[0])

    mask = torch.zeros_like(S, dtype=torch.bool)       # [F, T]
    mask[lo:hi, :] = True
    S_bp = torch.where(mask, S, torch.zeros_like(S))

    # iSTFT back to time (compat wrapper)
    y = _istft(S_bp, length=x.numel())                 # [T]
    y = y - y.mean()
    y = torch.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
    return y.unsqueeze(0)

In [ ]:
def spectral_subtract_quiet_frames(waveform: torch.Tensor, sr: int) -> torch.Tensor:
    x = waveform.squeeze(0)    # [T]
    Tlen = x.shape[-1]
    S = _stft(x)               # [F, T] complex
    Mag = S.abs()
    Pow = Mag**2

    freqs = np.fft.rfftfreq(NFFT, d=1.0/sr)
    lo = int(np.searchsorted(freqs, BAND_LO))
    hi = int(np.searchsorted(freqs, BAND_HI))
    lo = max(lo, 0); hi = min(hi, Mag.shape[0])

    band_pow_per_frame = Pow[lo:hi].mean(dim=0)         # [T_frames]
    T_frames = band_pow_per_frame.numel()
    k = max(1, int(round(QUIET_PCT * T_frames)))
    vals, idxs = torch.topk(-band_pow_per_frame, k)     # quietest frames
    quiet_mask = torch.zeros_like(band_pow_per_frame, dtype=torch.bool)
    quiet_mask[idxs] = True

    Npsd = Pow[:, quiet_mask].mean(dim=1, keepdim=True) # noise PSD
    Pclean = torch.clamp(Pow - OVERSUB * Npsd, min=0.0)
    Mag_clean = torch.sqrt(Pclean + 1e-12)
    S_clean = Mag_clean * torch.exp(1j * S.angle())
    y_clean = _istft(S_clean, length=Tlen)              # [T]
    y_clean = y_clean - y_clean.mean()
    y_clean = torch.nan_to_num(y_clean, nan=0.0, posinf=0.0, neginf=0.0)
    return y_clean.unsqueeze(0)

def clean_chunk(waveform: torch.Tensor, sr: int) -> torch.Tensor:
    if waveform.dim() == 2 and waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    elif waveform.dim() == 1:
        waveform = waveform.unsqueeze(0)
    if sr != SR:
        waveform = torchaudio.functional.resample(waveform, sr, SR)
        sr = SR
    y_bp = bandpass_chunk(waveform, sr)
    y_cl = spectral_subtract_quiet_frames(y_bp, sr)
    return y_cl

In [ ]:
# DATASET
class AudioImageDataset(Dataset):
    def __init__(self, audio_dir, image_dir, label_map, transform_image=None,
                 sample_rate=SAMPLE_RATE, n_mels=N_MELS, use_filters=True):
        self.audio_dir = audio_dir
        self.image_dir = image_dir
        self.label_map = label_map
        self.transform_image = transform_image
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.use_filters = use_filters

        self.audio_files, self.labels, self.video_ids = [], [], []
        for file in os.listdir(audio_dir):
            if file.endswith(".wav") and "_seg" in file:
                frame_file = file.replace("seg", "frame").replace(".wav", ".jpg")
                if not os.path.exists(os.path.join(image_dir, frame_file)):
                    continue
                base_name = file.split("_seg")[0]                
                label_str = base_name.split("_")[0].lower()
                if label_str in label_map:
                    self.audio_files.append(file)                 
                    self.labels.append(label_map[label_str])
                    self.video_ids.append(file)                   

        # Audio -> Mel
        self.mel = T.MelSpectrogram(sample_rate=self.sample_rate, n_fft=1024,
                                    hop_length=512, n_mels=self.n_mels)
        self.db  = T.AmplitudeToDB()

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]         
        label      = self.labels[idx]
        seg_id     = self.video_ids[idx]

        # ---- AUDIO ----
        a_path = os.path.join(self.audio_dir, audio_file)
        waveform, sr = torchaudio.load(a_path)     # 2s chunk
        if self.use_filters:
            waveform = clean_chunk(waveform, sr)   # [1, T] @ 48k
        else:
            if sr != self.sample_rate:
                waveform = torchaudio.functional.resample(waveform, sr, self.sample_rate)

        spec = self.mel(waveform)                  # [1, n_mels, time]
        spec = self.db(spec)
        spec = (spec - spec.mean()) / (spec.std() + 1e-6)
        spec = F.interpolate(spec.unsqueeze(0), size=(224,224), mode="bilinear", align_corners=False)
        spec = spec.mean(dim=1)                    # [1,224,224]

        # ---- IMAGE ----
        frame_file = audio_file.replace("seg", "frame").replace(".wav", ".jpg")
        img_path = os.path.join(self.image_dir, frame_file)
        img = Image.open(img_path).convert("RGB")
        if self.transform_image:
            img = self.transform_image(img)

        return spec, img, label, seg_id